In [1]:
try:
    import pandas as pd
    import time as time
    import database as db
    from bs4 import BeautifulSoup
    from unidecode import unidecode
    from selenium import webdriver
    from selenium.webdriver.common.by import By
    from selenium.webdriver.common.keys import Keys 
    from selenium.webdriver.chrome.options import Options
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
except Exception as e:
    print(e)

In [2]:
data = db.DataBase()

In [3]:
def getData(ano, formacao, inicio=0, limitePaginas=None, curriculo=1):
    op = Options()
    op.headless = True
    navegador = webdriver.Chrome(options=op)
    navegador.get("http://buscatextual.cnpq.br/buscatextual/busca.do")
    
    #Tags de pesquisa
    if formacao != "doutorado":
        navegador.find_element_by_xpath('//*[@id="buscarDemais"]').send_keys(Keys.SPACE) #Buscar demais pesquisadores

        if formacao == "mestrado":
            navegador.find_element_by_xpath('//*[@id="filtro2"]').click()
            time.sleep(1)
            navegador.find_element_by_xpath('//*[@id="nivelFormacao"]').click()
            navegador.find_element_by_xpath('//*[@id="nivelFormacao"]').send_keys(Keys.DOWN)
            navegador.find_element_by_xpath('//*[@id="nivelFormacao"]').send_keys(Keys.DOWN)
            navegador.find_element_by_xpath('//*[@id="nivelFormacao"]').send_keys(Keys.DOWN)
            navegador.find_element_by_xpath('//*[@id="nivelFormacao"]').send_keys(Keys.ENTER)
            time.sleep(1)
            navegador.find_element_by_xpath('//*[@id="preencheCategoriaNivelBolsa"]')

    navegador.find_element_by_xpath('//*[@id="buscarEstrangeiros"]').click()
            
    navegador.find_element_by_xpath('//*[@id="buscaAssunto"]').send_keys(Keys.SPACE) #Buscar por assunto

    #Caixa de texto
    buscar = navegador.find_element_by_xpath('//*[@id="textoBusca"]')
    buscar.send_keys(ano)
    buscar.send_keys(Keys.ENTER)

    time.sleep(2)

    passadas = []

    janelaAtual = navegador.current_window_handle
    passadas.append(navegador.window_handles)

    #Variaveis de controle da paginação
    pagina = 1
    pag = 2
    troca = True
    delay_max = 180
    
    if limitePaginas==None:
        WebDriverWait(navegador, delay_max).until(EC.presence_of_element_located((By.XPATH, '/html/body/form/div/div[4]/div/div/div/div[3]/div/div[1]/b[1]')))
        limitePaginas = int(navegador.find_element_by_xpath('/html/body/form/div/div[4]/div/div/div/div[3]/div/div[1]/b[1]').get_attribute("innerHTML").splitlines()[0])
        limitePaginas = int(paginasTotais/10 if paginasTotais%10==0 else (paginasTotais/10)+1)
    
    if inicio!=0:
        WebDriverWait(navegador, delay_max).until(EC.presence_of_element_located((By.XPATH, f'/html/body/form/div/div[4]/div/div/div/div[3]/div/div[4]/a[{pag}]')))
        navegador.find_element_by_xpath(f'/html/body/form/div/div[4]/div/div/div/div[3]/div/div[4]/a[{pag}]').click()
        pagina = inicio
        link = navegador.current_url.split(";")
        link[0] = link[0].replace(str(10), str((pagina-1)*10))
        link = link[0]+";"+link[1]

        pag = inicio+1
        if inicio<11:
            pag = inicio+1
        elif inicio%10==0:
            pag = 13
        else:
            div = "1"+("0"*(len(str(inicio))-1))
            pag = int((inicio+1)/int(div))+2
        navegador.get(link)
        troca = False
        inicio = 0
    
    while(True):
        #Troca de pagina        
        if troca:
            try:
                if pagina!=1 and pag<=11 and pagina<=11:
                    WebDriverWait(navegador, delay_max).until(EC.presence_of_element_located((By.XPATH, f'/html/body/form/div/div[4]/div/div/div/div[3]/div/div[4]/a[{pag}]')))
                    navegador.find_element_by_xpath(f'/html/body/form/div/div[4]/div/div/div/div[3]/div/div[4]/a[{pag}]').click()
                    pag = pag+1 if pag<11 else 3

                if pagina>11 and pag>2:
                    WebDriverWait(navegador, delay_max).until(EC.presence_of_element_located((By.XPATH, f'/html/body/form/div/div[4]/div/div/div/div[3]/div/div[4]/a[{pag}]')))
                    navegador.find_element_by_xpath(f'/html/body/form/div/div[4]/div/div/div/div[3]/div/div[4]/a[{pag}]').click()
                    pag = pag+1 if pag<13 else 3
            except Exception as e:
                print(e)
                break
       
        troca = True
        endFor = False
        for i in range(curriculo,11):
            try:
                #Selecionar curriculo
                WebDriverWait(navegador, delay_max).until(EC.presence_of_element_located((By.XPATH, f'//html/body/form/div/div[4]/div/div/div/div[3]/div/div[3]/ol/li[{i}]/b/a')))
                navegador.find_element_by_xpath(f'//html/body/form/div/div[4]/div/div/div/div[3]/div/div[3]/ol/li[{i}]/b/a').click()
                
                time.sleep(1)

                #Abrir curriculo
                WebDriverWait(navegador, delay_max).until(EC.presence_of_element_located((By.XPATH, '//*[@id="idbtnabrircurriculo"]')))
                navegador.find_element_by_xpath('//*[@id="idbtnabrircurriculo"]').click()
            
            except Exception as e:
                print(e)
                endFor = True
                break
                
            time.sleep(2)

            try:
                for janela in navegador.window_handles:
                    if janela not in passadas:
                        navegador.switch_to.window(janela)

            except:
                navegador.switch_to.window(janelaAtual)
                navegador.get(navegador.current_url)
                i -= 1
                continue
            
            try:
                #Pegar dados
                WebDriverWait(navegador, delay_max).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[3]/div/div/div/div[1]/h2'))) 
                site = BeautifulSoup(navegador.page_source, 'html.parser')
                artigos = []

                for artigo in site.findAll('div', attrs={'class':['artigo-completo']}):
                    if str(ano) in artigo.text:
                        artigo = artigo.text[10:-4:]
                        text = ""
                        maior = 0
                        for s in artigo.split("."):
                            for ss in s.split(","):
                                if len(ss) > maior:
                                    text = ss
                                    maior = len(ss)
                        artigos.append(text)

                nome = site.findAll('h2', attrs={'class':['nome']})[0].text
            except:
                time.sleep(1)
                i-=1
                continue
            
            #Inserir no banco de dados
            if artigos != []:
                data.insertData(nome, formacao, artigos, ano)

            navegador.close()

            time.sleep(1)

            navegador.switch_to.window(janelaAtual)

            #Fechar curriculo
            WebDriverWait(navegador, delay_max).until(EC.presence_of_element_located((By.XPATH, '//*[@id="idbtnfechar"]')))
            navegador.find_element_by_xpath('//*[@id="idbtnfechar"]').click()
        
        if endFor:
            break
        
        data.connection.commit()
        
        #Limitador de pagínas
        if pagina == limitePaginas:
            break
        
        pagina += 1
        curriculo = 1
        
    navegador.quit()
    if pagina == limitePaginas:
        return pagina, i
    else:
        return getData(ano, formacao, inicio=pagina, curriculo=i, limitePaginas=limitePaginas)

In [7]:
try:
    p18, i18 = getData("2018", "doutorado", limitePaginas=500)
except Exception as e:
    print(e)

Informações de pesquisa.
Message: 

Informações de pesquisa.
Message: element click intercepted: Element <a href="javascript:abreDetalhe('K4511707T2','Maria_Helena_Machado',1870239,)">...</a> is not clickable at point (126, 414). Other element would receive the click: <iframe scrolling="no" name="frameModalPreview" frameborder="0" class="iframe-modal" src="/buscatextual/preview.do?metodo=apresentar&amp;id=K8443549P9" style="height: 335px;"></iframe>
  (Session info: headless chrome=95.0.4638.54)

Informações de pesquisa.
Message: 

Informações de pesquisa.
Message: element click intercepted: Element <a data-role="paginacao" href="/buscatextual/busca.do?metodo=forwardPaginaResultados&amp;?numeroPagina=70&amp;registros=700;10&amp;query=%2Bidx_assunto%3A%282018%29++%2Bidx_particao%3A1+%2Bidx_nacionalidade%3Ab+%5E500&amp;analise=cv&amp;tipoOrdenacao=null&amp;paginaOrigem=index.do&amp;mostrarScore=true&amp;mostrarBandeira=false&amp;modoIndAdhoc=null">...</a> is not clickable at point (518, 

KeyboardInterrupt: 

In [5]:
data.selectData(2018)

,primeiro_nome,nome_completo,formacao,titulo,ano
